In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
from datetime import datetime
import time
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.metrics import accuracy_score,f1_score,recall_score,precision_score,confusion_matrix,ConfusionMatrixDisplay,roc_curve,roc_auc_score,precision_recall_curve
from sklearn.ensemble import RandomForestClassifier , StackingClassifier
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression,Lasso
from sklearn.preprocessing import Binarizer
from sklearn.model_selection import cross_val_score,GridSearchCV

# 한글 깨짐 방지
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'malgun Gothic'

In [16]:
train = pd.read_csv('./Dataset/train_Winsorization.csv', encoding='euc-kr')

test = pd.read_csv('./Dataset/test.csv', encoding='euc-kr')

In [17]:
# train['기업수명주기'] = train['기업수명주기'].map({
#     '도입기' : 1,
#     '성장기' : 2,
#     '성숙기' : 3,
#     '수축기' : 4,
#     '쇠퇴기' : 5
# }).astype('category')

# test['기업수명주기'] = test['기업수명주기'].map({
#     '도입기' : 1,
#     '성장기' : 2,
#     '성숙기' : 3,
#     '수축기' : 4,
#     '쇠퇴기' : 5
# }).astype('category')

# train['파부비초과여부']=train['파부비초과여부'].astype('category')
# test['파부비초과여부']=test['파부비초과여부'].astype('category')
# train['파차의초과여부']=train['파차의초과여부'].astype('category')
# test['파차의초과여부']=test['파차의초과여부'].astype('category')

In [18]:
X_train_sum=train[['유동자산회전률', '총자산대비잉여현금흐름','자기자본구성비율', 'log자산총계','자기자본회전률', '순운전자본회전률', '자기자본증가율', '총자본증가율', '총자산대비현금흐름', '총자본투자효율']]
y_train = train[['t-1감사의견코드']]
X_test_sum=test[['유동자산회전률', '총자산대비잉여현금흐름','자기자본구성비율', 'log자산총계','자기자본회전률', '순운전자본회전률', '자기자본증가율', '총자본증가율', '총자산대비현금흐름', '총자본투자효율']]
y_test = test[['t-1감사의견코드']]

In [19]:
X_train = train.drop('t-1감사의견코드',axis=1)
y_train = train[['t-1감사의견코드']]

X_test = test.drop('t-1감사의견코드',axis=1)
y_test = test[['t-1감사의견코드']]

In [20]:
scaler = StandardScaler()
X_train_sc = scaler.fit_transform(X_train_sum)
X_test_sc = scaler.transform(X_test_sum)

In [21]:
X_train_sc=pd.DataFrame(X_train_sc, columns=X_train_sum.columns)
X_test_sc=pd.DataFrame(X_test_sc, columns=X_test_sum.columns)

In [22]:
from imblearn.under_sampling import OneSidedSelection
from imblearn.under_sampling import RandomUnderSampler

#
# OneSidedSelection 적용
undersampler = OneSidedSelection(random_state=42)
X_train_sc, y_train = undersampler.fit_resample(X_train_sc, y_train)

# 1:1 비율로 데이터셋을 다시 조정합니다.
custom_undersampler = RandomUnderSampler(sampling_strategy='majority', random_state=42)
X_train_sc, y_train = custom_undersampler.fit_resample(X_train_sc, y_train)


In [23]:
from imblearn.under_sampling import OneSidedSelection
from imblearn.under_sampling import RandomUnderSampler

# OneSidedSelection 적용
undersampler = OneSidedSelection(random_state=42)
X_test_sc, y_test = undersampler.fit_resample(X_test_sc, y_test)

# 1:1 비율로 데이터셋을 다시 조정합니다.
custom_undersampler = RandomUnderSampler(sampling_strategy='majority', random_state=42)
X_test_sc, y_test = custom_undersampler.fit_resample(X_test_sc, y_test)


In [24]:
X_train_sc=pd.concat([X_train_sc,y_train],axis=1)

In [25]:
X_test_sc=pd.concat([X_test_sc,y_test],axis=1)

In [26]:
X_train_sc['t-1감사의견코드'].value_counts()

0.0    9602
1.0    9602
Name: t-1감사의견코드, dtype: int64

In [13]:
X_test_sc['t-1감사의견코드'].value_counts()

0.0    2868
1.0    2868
Name: t-1감사의견코드, dtype: int64

------ 여기까지 보기 --------

In [31]:
# X_test_sc.to_csv('test_1대1.csv',index=False,encoding='euc-kr')
# X_train_sc.to_csv('train_1대1.csv',index=False,encoding='euc-kr')

---

In [80]:
X_test_sum

,총자본증가율,총자산대비잉여현금흐름,당좌자산회전률,순운전자본회전률,자기자본회전률,log자산총계,자기자본증가율,총자산대비현금흐름,매출액대비잉여현금흐름,영업이익증가율,총자본투자효율,총자본순이익률,기업수명주기,파부비초과여부,파차의초과여부
0,-0.170247,0.137782,-0.415536,-0.019814,-0.332972,0.159766,-0.125789,-0.108062,0.110256,-0.368459,-0.285538,0.051811,3,0,0
1,-0.207772,0.060285,0.692143,-0.916968,-0.407785,-0.439209,-0.202516,-0.126357,0.082347,-0.435283,-0.549591,-0.238997,1,0,0
2,-0.595396,0.632466,0.021951,0.233729,-0.241193,-0.470121,-0.217648,0.383005,0.112923,-0.209201,-0.438607,-0.308464,3,0,0
3,-1.536126,2.148786,-0.317800,-0.325367,-0.193374,-0.576999,-0.072116,-0.143747,0.127368,-0.173910,-0.011802,-0.017656,4,0,0
4,0.114186,0.673176,-0.384509,0.012691,-0.300580,2.016369,-0.116923,0.624976,0.114803,-0.251926,0.055608,0.475422,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41120,-0.083191,0.157306,-0.167317,-0.143336,-0.305978,0.934949,-0.091150,-0.052811,0.110563,-0.219535,-0.205093,0.175080,3,0,0
41121,-0.247172,0.394814,-0.702540,-0.877961,-0.241964,0.167218,-0.118696,-0.121818,0.131813,-0.421191,-0.305277,-0.214479,3,0,0
41122,0.319818,-0.222172,0.882962,-0.806449,0.011009,-0.663137,-0.158655,0.216852,0.099715,-0.173910,-0.436000,-0.224695,5,1,1
41123,-0.099327,0.228863,-0.409331,-0.858458,0.415920,0.324669,-0.181117,-0.121857,0.124452,-0.206342,-0.427062,-0.238997,3,1,0


- X_train_sc, X_test
- y_train_sc ,y_test

In [82]:
from imblearn.over_sampling import SMOTENC
import pandas as pd
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier

def model_basic(x_train, y_train, x_test, y_test, sampling_ratios):
    categorical_features_indices = [12, 13, 14]

    models = [
        LogisticRegression(C=0.01, penalty='l2', random_state=0),
        RandomForestClassifier(random_state=0),
        LGBMClassifier(random_state=0),
        CatBoostClassifier(random_state=0,cat_features=categorical_features_indices,verbose=False) ,
        SVC(random_state=0)
    ]

    rdict = {
        'model': [],
        'sampling_ratio': [],
        'acc_train': [],
        'auc_train': [],
        'acc_test': [],
        'precision': [],
        'recall': [],
        'f1_score': [],
        'AUC_test': []
    }

    for ratio in sampling_ratios:
        smote_nc_train = SMOTENC(categorical_features=categorical_features_indices, sampling_strategy=ratio)
        smote_nc_test = SMOTENC(categorical_features=categorical_features_indices, sampling_strategy=ratio)
        X_resampled_train, y_resampled_train = smote_nc_train.fit_resample(x_train, y_train)
        X_resampled_test, y_resampled_test = smote_nc_test.fit_resample(x_test, y_test)

        for clf in models:
            clf = clf.fit(X_resampled_train, y_resampled_train)

            # Train
            y_hat_train = clf.predict(X_resampled_train)
            results_train = (round(accuracy_score(y_resampled_train, y_hat_train), 4), round(roc_auc_score(y_resampled_train, y_hat_train), 4))

            # Test
            y_hat_test = clf.predict(X_resampled_test)
            results = (round(accuracy_score(y_resampled_test, y_hat_test), 4),
                       round(precision_score(y_resampled_test, y_hat_test), 4),
                       round(recall_score(y_resampled_test, y_hat_test), 4),
                       round(f1_score(y_resampled_test, y_hat_test), 4),
                       round(roc_auc_score(y_resampled_test, y_hat_test), 4))

            rdict['model'].append(clf)
            rdict['sampling_ratio'].append(ratio)
            rdict['acc_train'].append(results_train[0])
            rdict['auc_train'].append(results_train[1])
            rdict['acc_test'].append(results[0])
            rdict['precision'].append(results[1])
            rdict['recall'].append(results[2])
            rdict['f1_score'].append(results[3])
            rdict['AUC_test'].append(results[4])

            confusion = confusion_matrix(y_resampled_test, y_hat_test)
            print(f"Sampling Ratio: {ratio}")
            print(confusion)

    rdf_final = pd.DataFrame(data=rdict)
    return rdf_final

In [84]:
# 함수 호출
sampling_ratios = [0.1,0.2, 0.3,0.4, 0.5, 0.6,0.7,0.8,0.9,1]
result_dataframe = model_basic(X_train_sum, y_train, X_test_sum, y_test, sampling_ratios)
result_dataframe


Sampling Ratio: 0.1
[[37607   650]
 [ 3259   566]]
Sampling Ratio: 0.1
[[37376   881]
 [ 1958  1867]]
Sampling Ratio: 0.1
[[37161  1096]
 [ 1736  2089]]
Sampling Ratio: 0.1
[[37223  1034]
 [ 1748  2077]]
Sampling Ratio: 0.1
[[37659   598]
 [ 2659  1166]]
Sampling Ratio: 0.2
[[36744  1513]
 [ 4998  2653]]
Sampling Ratio: 0.2
[[36923  1334]
 [ 2879  4772]]
Sampling Ratio: 0.2
[[36420  1837]
 [ 2306  5345]]
Sampling Ratio: 0.2
[[36620  1637]
 [ 2382  5269]]
Sampling Ratio: 0.2
[[36807  1450]
 [ 3419  4232]]
Sampling Ratio: 0.3
[[35993  2264]
 [ 5725  5752]]
Sampling Ratio: 0.3
[[36655  1602]
 [ 3798  7679]]
Sampling Ratio: 0.3
[[35988  2269]
 [ 3006  8471]]
Sampling Ratio: 0.3
[[36190  2067]
 [ 3014  8463]]
Sampling Ratio: 0.3
[[36237  2020]
 [ 4177  7300]]
Sampling Ratio: 0.4
[[35332  2925]
 [ 6209  9093]]
Sampling Ratio: 0.4
[[36466  1791]
 [ 4558 10744]]
Sampling Ratio: 0.4
[[35571  2686]
 [ 3493 11809]]
Sampling Ratio: 0.4
[[35928  2329]
 [ 3507 11795]]
Sampling Ratio: 0.4
[[35817  24

,model,sampling_ratio,acc_train,auc_train,acc_test,precision,recall,f1_score,AUC_test
0,"LogisticRegression(C=0.01, random_state=0)",0.1,0.9090,0.5679,0.9071,0.4655,0.1480,0.2246,0.5655
1,"(DecisionTreeClassifier(max_features='sqrt', r...",0.1,1.0000,0.9998,0.9325,0.6794,0.4881,0.5681,0.7325
2,LGBMClassifier(random_state=0),0.1,0.9390,0.7846,0.9327,0.6559,0.5461,0.5960,0.7587
3,<catboost.core.CatBoostClassifier object at 0x...,0.1,0.9525,0.8258,0.9339,0.6676,0.5430,0.5989,0.7580
4,SVC(random_state=0),0.1,0.9259,0.6663,0.9226,0.6610,0.3048,0.4172,0.6446
5,"LogisticRegression(C=0.01, random_state=0)",0.2,0.8602,0.6560,0.8582,0.6368,0.3468,0.4490,0.6536
6,"(DecisionTreeClassifier(max_features='sqrt', r...",0.2,1.0000,1.0000,0.9082,0.7815,0.6237,0.6938,0.7944
7,LGBMClassifier(random_state=0),0.2,0.9172,0.8402,0.9098,0.7442,0.6986,0.7207,0.8253
8,<catboost.core.CatBoostClassifier object at 0x...,0.2,0.9366,0.8785,0.9125,0.7630,0.6887,0.7239,0.8229
9,SVC(random_state=0),0.2,0.9012,0.7815,0.8939,0.7448,0.5531,0.6348,0.7576


In [85]:
# result_dataframe.to_csv('외감_오버샘플링_비율.csv',index=False)